## Goal of the Competition
In this “getting started” competition, you’ll use time-series forecasting to forecast store sales on data from Corporación Favorita, a large Ecuadorian-based grocery retailer.

Specifically, you'll build a model that more accurately predicts the unit sales for thousands of items sold at different Favorita stores. You'll practice your machine learning skills with an approachable training dataset of dates, store, and item information, promotions, and unit sales.

Get Started
We highly recommend the "Time Series course" , which walks you through how to make your first submission. The lessons in this course are inspired by winning solutions from past Kaggle time series forecasting competitions.

## Dataset Description
In this competition, you will predict sales for the thousands of product families sold at Favorita stores located in Ecuador. The training data includes dates, store and product information, whether that item was being promoted, as well as the sales numbers. Additional files include supplementary information that may be useful in building your models.

File Descriptions and Data Field Information
#### train.csv
The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.
store_nbr identifies the store at which the products are sold.
family identifies the type of product sold.
sales gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.
####  test.csv
The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
The dates in the test data are for the 15 days after the last date in the training data.
#### sample_submission.csv
A sample submission file in the correct format.
#### stores.csv
Store metadata, including city, state, type, and cluster.
cluster is a grouping of similar stores.
#### oil.csv
Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)
#### holidays_events.csv
Holidays and Events, with metadata
NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).
#### Additional Notes
Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'ADNAN SHIKH ALHADDADEEN---ExploratoryDataAnalysis.ipynb',
 'final_data.csv',
 'holidays_events.csv',
 'main.ipynb',
 'oil.csv',
 'sample_submission.csv',
 'stores.csv',
 'submission1.csv',
 'submission2.csv',
 'submission3XGBoost.csv',
 'submission4RandomForestRegressor.csv',
 'test.csv',
 'test_final.csv',
 'train.csv',
 'train_final.csv',
 'transactions.csv',
 'Untitled.ipynb']

# 1.Viewing The Data

In [3]:
oil = pd.read_csv("oil.csv")
train = pd.read_csv("train.csv")
stores = pd.read_csv("stores.csv")
transactions = pd.read_csv("transactions.csv")
holidays = pd.read_csv("holidays_events.csv")

In [4]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [5]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [6]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [7]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [8]:
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


# 2.Combining All The Datasets into Train Dataset
### Common Features:
* holidays dataset has **date** column
* oil dataset has **date** column
* stores dataset has **store_nbr** column
* transactions dataset has both **date, and store_nbr** column

In [9]:
train = pd.merge(train, holidays, on="date", how="left")

In [10]:
train

,id,date,store_nbr,family,sales,onpromotion,type,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False
...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,Local,Riobamba,Fundacion de Riobamba,False


In [11]:
train = pd.merge(train, stores, on="store_nbr", how="left")
train = pd.merge(train, transactions, on=["store_nbr", "date"], how="left")
train.head()

,id,date,store_nbr,family,sales,onpromotion,type_x,locale,locale_name,description,transferred,city,state,type_y,cluster,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN


In [12]:
#type_x is holiday_type
#type_y is store_type
train.rename(columns={"type_x":"holiday_type", "type_y":"store_type"}, inplace=True)
train

,id,date,store_nbr,family,sales,onpromotion,holiday_type,locale,locale_name,description,transferred,city,state,store_type,cluster,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
1,1,2013-01-01,1,BABY CARE,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
2,2,2013-01-01,1,BEAUTY,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
3,3,2013-01-01,1,BEVERAGES,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
4,4,2013-01-01,1,BOOKS,0.000,0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,Quito,Pichincha,B,6,2155.0
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Holiday,Local,Riobamba,Fundacion de Riobamba,False,Quito,Pichincha,B,6,2155.0
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Holiday,Local,Riobamba,Fundacion de Riobamba,False,Quito,Pichincha,B,6,2155.0
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Holiday,Local,Riobamba,Fundacion de Riobamba,False,Quito,Pichincha,B,6,2155.0


# 3.Seperating features into numerical(discrete and continious) and categorical features

In [13]:
#dropping the id column after making it into index
train.set_index("id")
train.drop("id", axis=1, inplace=True)

In [14]:
def seperate_category_types(df):
    discrete_numerical_features = []
    continuous_nuemrical_features = []
    categorical_features = []#contains string type values.
    #iterating over df's items.
    for feature, value in df.items():
        #if the value in that feature column is string-->>
        if pd.api.types.is_string_dtype(value):
            categorical_features.append(feature)
        #if the value in that feature column is numerical-->>
        elif pd.api.types.is_numeric_dtype(value):
            len_of_unique_values = len(train[feature].unique())
            #if there are more than 100 distinct values for this feature-->>
            if len_of_unique_values < 100: 
                discrete_numerical_features.append(feature)
            #if not-->>
            else:
                continuous_nuemrical_features.append(feature)
    return discrete_numerical_features, continuous_nuemrical_features, categorical_features


In [15]:
discrete_numerical_features, continuous_nuemrical_features, categorical_features = seperate_category_types(train)


In [16]:
discrete_numerical_features, continuous_nuemrical_features, categorical_features

(['store_nbr', 'cluster'],
 ['sales', 'onpromotion', 'transactions'],
 ['date',
  'family',
  'holiday_type',
  'locale',
  'locale_name',
  'description',
  'transferred',
  'city',
  'state',
  'store_type'])

# 4.Feature engineering

### 4.1.1 Now we've made our features into 3 different category of features.Let's see if there are missing values in categorical_features and fill them -->>

In [17]:
#these 5 columns are the columns of the holidays dataset.
#if they are missing it basically means there was no holiday or event on that date 
#so we can encode them with "missing" keyword
train[categorical_features].isna().sum()

date                  0
family                0
holiday_type    2551824
locale          2551824
locale_name     2551824
description     2551824
transferred     2551824
city                  0
state                 0
store_type            0
dtype: int64

In [18]:
for feature in holidays.columns:
    if feature != "date":
        if feature == "type":
            feature = "holiday_type"
        train[feature].fillna("missing", inplace=True)

In [19]:
#we've succesfully filled the missing values
train[categorical_features].isna().sum()

date            0
family          0
holiday_type    0
locale          0
locale_name     0
description     0
transferred     0
city            0
state           0
store_type      0
dtype: int64

### 4.1.2 Let's see if there are missing values in discrete_numerical_features and fill them-->>
* There aren't any missing values in discrete_numerical_features

In [20]:
train[discrete_numerical_features].isna().sum()

store_nbr    0
cluster      0
dtype: int64

### 4.1.3 Let's see if there are missing values in continious_numerical_features -->>

In [21]:
#transactions column has missing values let's see if there were no transactions
#would there be any sales
train[continuous_nuemrical_features].isna().sum()

sales                0
onpromotion          0
transactions    249117
dtype: int64

* we can see here that if no transactions made for that particular day; there is %98 chance that the sales will be 0, compared to whole dataset which has only has 31% chance of sales being 0

**So we can say that for days with no transactions, sales tend to be 0**

In [22]:
count_of_days_that_sales_are_0 = train.loc[train.transactions.isna(), "sales"].value_counts()[0]
print(f"{(count_of_days_that_sales_are_0 / len(train[train.transactions.isna()])) * 100}% of sales are 0")

98.69619496060085% of sales are 0


In [23]:
count_of_days_that_sales_are_0 = train.sales.value_counts()[0]
print(f"{(count_of_days_that_sales_are_0 / len(train)) * 100}% of sales are 0")

31.22568220779034% of sales are 0


In [24]:
count_of_days_that_sales_are_0 = train.loc[train.transactions.dropna().index, "sales"].value_counts()[0]
print(f"{(count_of_days_that_sales_are_0 / len(train.transactions.dropna().index)) * 100}% of sales are 0")

25.23400033722713% of sales are 0


### Let's examine if there is no transaction why there are sales for some days

* Hyphotesis: most of the times if there aren't any transactions it means there were no sales made that day, for the days there are sales but also no transaction's made it indicates that transactions weren't recorded

* We are going to fill nan transaction values with the value 0.

In [25]:
train.transactions.fillna(0, inplace=True)

### 5.Now we've filled the missing data, it's time to encode categorical variables into a meaningful numeric format.

In [36]:
filled_train = train.copy()
filled_train.to_csv("filled_train.csv", index=False)
del filled_train

In [98]:
train.isna().sum()

date            0
store_nbr       0
family          0
sales           0
onpromotion     0
holiday_type    0
locale          0
locale_name     0
description     0
transferred     0
city            0
state           0
store_type      0
cluster         0
transactions    0
dtype: int64

In [99]:
train[categorical_features]

,date,family,holiday_type,locale,locale_name,description,transferred,city,state,store_type
0,2013-01-01,AUTOMOTIVE,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D
1,2013-01-01,BABY CARE,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D
2,2013-01-01,BEAUTY,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D
3,2013-01-01,BEVERAGES,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D
4,2013-01-01,BOOKS,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D
...,...,...,...,...,...,...,...,...,...,...
3054343,2017-08-15,POULTRY,Holiday,Local,Riobamba,Fundacion de Riobamba,False,Quito,Pichincha,B
3054344,2017-08-15,PREPARED FOODS,Holiday,Local,Riobamba,Fundacion de Riobamba,False,Quito,Pichincha,B
3054345,2017-08-15,PRODUCE,Holiday,Local,Riobamba,Fundacion de Riobamba,False,Quito,Pichincha,B
3054346,2017-08-15,SCHOOL AND OFFICE SUPPLIES,Holiday,Local,Riobamba,Fundacion de Riobamba,False,Quito,Pichincha,B


In [100]:
for cat_feature in categorical_features:
    print(f"{cat_feature} : {train[cat_feature].nunique()} distinct values")

date : 1684 distinct values
family : 33 distinct values
holiday_type : 7 distinct values
locale : 4 distinct values
locale_name : 25 distinct values
description : 102 distinct values
transferred : 3 distinct values
city : 22 distinct values
state : 16 distinct values
store_type : 5 distinct values


### 5.1 Let's prepare the test data.

In [29]:
test = pd.read_csv("test.csv")
test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [30]:
test = pd.merge(test, holidays, on="date", how="left")
test = pd.merge(test, stores, on="store_nbr", how="left")
test = pd.merge(test, transactions, on=["store_nbr", "date"], how="left")
test.rename(columns={"type_x":"holiday_type", "type_y":"store_type"}, inplace=True)
test

,id,date,store_nbr,family,onpromotion,holiday_type,locale,locale_name,description,transferred,city,state,store_type,cluster,transactions
0,3000888,2017-08-16,1,AUTOMOTIVE,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,NaN
1,3000889,2017-08-16,1,BABY CARE,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,NaN
2,3000890,2017-08-16,1,BEAUTY,2,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,NaN
3,3000891,2017-08-16,1,BEVERAGES,20,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,NaN
4,3000892,2017-08-16,1,BOOKS,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,B,6,NaN
28508,3029396,2017-08-31,9,PREPARED FOODS,0,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,B,6,NaN
28509,3029397,2017-08-31,9,PRODUCE,1,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,B,6,NaN
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,B,6,NaN


In [31]:
for feature in holidays.columns:
    if feature != "date":
        if feature == "type":
            feature = "holiday_type"
        test[feature].fillna("missing", inplace=True)    
test[categorical_features].isna().sum()

date            0
family          0
holiday_type    0
locale          0
locale_name     0
description     0
transferred     0
city            0
state           0
store_type      0
dtype: int64

In [32]:
test.transactions.fillna(0, inplace=True)

In [33]:
test.isna().sum()

id              0
date            0
store_nbr       0
family          0
onpromotion     0
holiday_type    0
locale          0
locale_name     0
description     0
transferred     0
city            0
state           0
store_type      0
cluster         0
transactions    0
dtype: int64

In [35]:
filled_test = test.copy()
filled_test.to_csv("filled_test.csv", index=False)
del filled_test

### 5.2 Let's check if the categorical features in test data have the same distinct values as in train data

In [106]:
for cat_feature in categorical_features:
    if cat_feature != "date":
        print(f"{cat_feature}:{all(item in train[cat_feature].unique() for item in test[cat_feature].unique())}")

family:True
holiday_type:True
locale:True
locale_name:True
description:True
transferred:True
city:True
state:True
store_type:True


* Since train and test categorical features have the same distinct categories we can get the mean of each category in train data and map them to test data.

In [107]:
for cat_feature in categorical_features:
    if cat_feature != "date":
        target_encoded_values = {}
        for cat in train[cat_feature].unique():
            target_encoded_values[cat] = train.loc[train[cat_feature] == cat, "sales"].median()
        train[cat_feature] = train[cat_feature].map(target_encoded_values)
        test[cat_feature] = test[cat_feature].map(target_encoded_values)

In [108]:
train

,date,store_nbr,family,sales,onpromotion,holiday_type,locale,locale_name,description,transferred,city,state,store_type,cluster,transactions
0,2013-01-01,1,5.0000,0.000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0
1,2013-01-01,1,0.0000,0.000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0
2,2013-01-01,1,2.0000,0.000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0
3,2013-01-01,1,1791.0000,0.000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0
4,2013-01-01,1,0.0000,0.000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,2017-08-15,9,206.1680,438.133,0,10.0,12.0,10.0,12.0,12.0,24.0,23.226,7.0,6,2155.0
3054344,2017-08-15,9,64.0000,154.553,1,10.0,12.0,10.0,12.0,12.0,24.0,23.226,7.0,6,2155.0
3054345,2017-08-15,9,405.9775,2419.729,148,10.0,12.0,10.0,12.0,12.0,24.0,23.226,7.0,6,2155.0
3054346,2017-08-15,9,0.0000,121.000,8,10.0,12.0,10.0,12.0,12.0,24.0,23.226,7.0,6,2155.0


In [109]:
test

,id,date,store_nbr,family,onpromotion,holiday_type,locale,locale_name,description,transferred,city,state,store_type,cluster,transactions
0,3000888,2017-08-16,1,5.0000,0,11.0,11.0,11.0,11.0,11.0,24.0,23.226,16.0,13,0.0
1,3000889,2017-08-16,1,0.0000,0,11.0,11.0,11.0,11.0,11.0,24.0,23.226,16.0,13,0.0
2,3000890,2017-08-16,1,2.0000,2,11.0,11.0,11.0,11.0,11.0,24.0,23.226,16.0,13,0.0
3,3000891,2017-08-16,1,1791.0000,20,11.0,11.0,11.0,11.0,11.0,24.0,23.226,16.0,13,0.0
4,3000892,2017-08-16,1,0.0000,0,11.0,11.0,11.0,11.0,11.0,24.0,23.226,16.0,13,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,206.1680,1,11.0,11.0,11.0,11.0,11.0,24.0,23.226,7.0,6,0.0
28508,3029396,2017-08-31,9,64.0000,0,11.0,11.0,11.0,11.0,11.0,24.0,23.226,7.0,6,0.0
28509,3029397,2017-08-31,9,405.9775,1,11.0,11.0,11.0,11.0,11.0,24.0,23.226,7.0,6,0.0
28510,3029398,2017-08-31,9,0.0000,9,11.0,11.0,11.0,11.0,11.0,24.0,23.226,7.0,6,0.0


### Now we have our data in numerical format, there is one thing left to do.We should extract meaning from the dates(day_of_week,week_of_month, month_of_year, year)
* 4 features will be derived from the datetime feature (day_of_week,week_of_month, month_of_year, year)

In [110]:
train.date = pd.to_datetime(train["date"])

In [111]:
test.date = pd.to_datetime(test["date"])

In [112]:
train["day_of_week"] = [date.day_of_week for date in train["date"]]
train["week_of_month"] = [(date.day - 1) // 7 + 1 for date in train["date"]]
train["month_of_year"] = [date.month for date in train["date"]]
train["year"] = [date.year for date in train["date"]]

In [113]:
test["day_of_week"] = [date.day_of_week for date in test["date"]]
test["week_of_month"] = [(date.day - 1) // 7 + 1 for date in test["date"]]
test["month_of_year"] = [date.month for date in test["date"]]
test["year"] = [date.year for date in test["date"]]

In [115]:
train

,date,store_nbr,family,sales,onpromotion,holiday_type,locale,locale_name,description,transferred,city,state,store_type,cluster,transactions,day_of_week,week_of_month,month_of_year,year
0,2013-01-01,1,5.0000,0.000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0,1,1,1,2013
1,2013-01-01,1,0.0000,0.000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0,1,1,1,2013
2,2013-01-01,1,2.0000,0.000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0,1,1,1,2013
3,2013-01-01,1,1791.0000,0.000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0,1,1,1,2013
4,2013-01-01,1,0.0000,0.000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0,1,1,1,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,2017-08-15,9,206.1680,438.133,0,10.0,12.0,10.0,12.0,12.0,24.0,23.226,7.0,6,2155.0,1,3,8,2017
3054344,2017-08-15,9,64.0000,154.553,1,10.0,12.0,10.0,12.0,12.0,24.0,23.226,7.0,6,2155.0,1,3,8,2017
3054345,2017-08-15,9,405.9775,2419.729,148,10.0,12.0,10.0,12.0,12.0,24.0,23.226,7.0,6,2155.0,1,3,8,2017
3054346,2017-08-15,9,0.0000,121.000,8,10.0,12.0,10.0,12.0,12.0,24.0,23.226,7.0,6,2155.0,1,3,8,2017


In [116]:
test

,id,date,store_nbr,family,onpromotion,holiday_type,locale,locale_name,description,transferred,city,state,store_type,cluster,transactions,day_of_week,week_of_month,month_of_year,year
0,3000888,2017-08-16,1,5.0000,0,11.0,11.0,11.0,11.0,11.0,24.0,23.226,16.0,13,0.0,2,3,8,2017
1,3000889,2017-08-16,1,0.0000,0,11.0,11.0,11.0,11.0,11.0,24.0,23.226,16.0,13,0.0,2,3,8,2017
2,3000890,2017-08-16,1,2.0000,2,11.0,11.0,11.0,11.0,11.0,24.0,23.226,16.0,13,0.0,2,3,8,2017
3,3000891,2017-08-16,1,1791.0000,20,11.0,11.0,11.0,11.0,11.0,24.0,23.226,16.0,13,0.0,2,3,8,2017
4,3000892,2017-08-16,1,0.0000,0,11.0,11.0,11.0,11.0,11.0,24.0,23.226,16.0,13,0.0,2,3,8,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,206.1680,1,11.0,11.0,11.0,11.0,11.0,24.0,23.226,7.0,6,0.0,3,5,8,2017
28508,3029396,2017-08-31,9,64.0000,0,11.0,11.0,11.0,11.0,11.0,24.0,23.226,7.0,6,0.0,3,5,8,2017
28509,3029397,2017-08-31,9,405.9775,1,11.0,11.0,11.0,11.0,11.0,24.0,23.226,7.0,6,0.0,3,5,8,2017
28510,3029398,2017-08-31,9,0.0000,9,11.0,11.0,11.0,11.0,11.0,24.0,23.226,7.0,6,0.0,3,5,8,2017


In [117]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3054348 entries, 0 to 3054347
Data columns (total 19 columns):
 #   Column         Dtype         
---  ------         -----         
 0   date           datetime64[ns]
 1   store_nbr      int64         
 2   family         float64       
 3   sales          float64       
 4   onpromotion    int64         
 5   holiday_type   float64       
 6   locale         float64       
 7   locale_name    float64       
 8   description    float64       
 9   transferred    float64       
 10  city           float64       
 11  state          float64       
 12  store_type     float64       
 13  cluster        int64         
 14  transactions   float64       
 15  day_of_week    int64         
 16  week_of_month  int64         
 17  month_of_year  int64         
 18  year           int64         
dtypes: datetime64[ns](1), float64(11), int64(7)
memory usage: 530.6 MB


In [118]:
for col in train.columns:
    if train[col].dtype == np.int64:
        train[col] = train[col].astype(np.int32)
    elif train[col].dtype == np.float64:
        train[col] = train[col].astype(np.float32)

### Reducing memory usage is good practice

In [119]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3054348 entries, 0 to 3054347
Data columns (total 19 columns):
 #   Column         Dtype         
---  ------         -----         
 0   date           datetime64[ns]
 1   store_nbr      int32         
 2   family         float32       
 3   sales          float32       
 4   onpromotion    int32         
 5   holiday_type   float32       
 6   locale         float32       
 7   locale_name    float32       
 8   description    float32       
 9   transferred    float32       
 10  city           float32       
 11  state          float32       
 12  store_type     float32       
 13  cluster        int32         
 14  transactions   float32       
 15  day_of_week    int32         
 16  week_of_month  int32         
 17  month_of_year  int32         
 18  year           int32         
dtypes: datetime64[ns](1), float32(11), int32(7)
memory usage: 320.8 MB


In [120]:
train = pd.get_dummies(train, columns=["day_of_week"], drop_first=True)
test =  pd.get_dummies(test, columns=["day_of_week"], drop_first=True)

In [123]:
X = train.drop(["date", "sales", "week_of_month","month_of_year","year"], axis=1)
y = train.sales

In [124]:
X

,store_nbr,family,onpromotion,holiday_type,locale,locale_name,description,transferred,city,state,store_type,cluster,transactions,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
0,1,5.000000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0,1,0,0,0,0,0
1,1,0.000000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0,1,0,0,0,0,0
2,1,2.000000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0,1,0,0,0,0,0
3,1,1791.000000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0,1,0,0,0,0,0
4,1,0.000000,0,10.0,13.0,13.0,0.0,12.0,24.0,23.226,16.0,13,0.0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,9,206.167999,0,10.0,12.0,10.0,12.0,12.0,24.0,23.226,7.0,6,2155.0,1,0,0,0,0,0
3054344,9,64.000000,1,10.0,12.0,10.0,12.0,12.0,24.0,23.226,7.0,6,2155.0,1,0,0,0,0,0
3054345,9,405.977509,148,10.0,12.0,10.0,12.0,12.0,24.0,23.226,7.0,6,2155.0,1,0,0,0,0,0
3054346,9,0.000000,8,10.0,12.0,10.0,12.0,12.0,24.0,23.226,7.0,6,2155.0,1,0,0,0,0,0


In [149]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [150]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_jobs=-1)
#!pip install xgboost
#from xgboost import XGBRegressor
#model = XGBRegressor()

In [151]:
%%time
model.fit(X_train, y_train)

CPU times: total: 35min 54s
Wall time: 6min 31s


RandomForestRegressor(n_jobs=-1)

In [152]:
model.score(X_test, y_test)

0.9041207179640167

In [153]:
#test.drop(["week_of_month","month_of_year","year"], axis=1, inplace=True)

In [154]:
preds = model.predict(test.drop(["date", "id"], axis=1))

In [155]:
predictions = pd.DataFrame({"id": test.id, "sales": preds})

In [156]:
predictions.loc[predictions.sales < 0, "sales"] = 0
predictions

,id,sales
0,3000888,2.899167
1,3000889,0.958667
2,3000890,21.510000
3,3000891,2552.510000
4,3000892,0.958667
...,...,...
28507,3029395,528.966840
28508,3029396,89.137950
28509,3029397,4800.941781
28510,3029398,112.520000


In [157]:
predictions.to_csv("submission4RandomForestRegressor.csv", index=False)

In [ ]:
train.to_csv("train_final.csv", index=False)
test.to_csv("test_final.csv", index=False)